In [1]:
from refexps import load_data, move_level, gamerun_level, get_previous #, lex_sim
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import compound_splitter
from scipy import stats
from nltk import word_tokenize
from string import punctuation
punct = list(punctuation)

In [2]:
def exp_len(exp, language, splits, compound):
    '''
    Extract content words by the phrase
    :param exp: referring expression tuple
    :return: set of content words of the phrase
    '''
    exp = exp[2]
    words = word_tokenize(exp, language)
    if compound == True:
        if language == 'german':
            words_nocomp = []
            for w in words:
                word = compound_splitter.split_word(w, splits)
                words_nocomp = words_nocomp + (word.split(' '))	
                words = words_nocomp
    return len(words)

def len_div(exp1, exp2, language, splits, compound):
    '''
    compute dice similarity between the content words in two referring expression
    :param exp1: referring expression tuple 1
    :param exp2: referring expression tuple 2
    :return: dice similarity between the content words in the expressions; undefined if they have none
    '''        
    len1 = exp_len(exp1, language, splits, compound)    
    len2 = exp_len(exp2, language, splits, compound)
    if exp1 is 'no_previous' or exp2 is 'no_previous':
        return 'undefined'
    if len1 + len2 == 0:
        return 'undefined'
    metric = float(len1 - len2)/float(len1 + len2)
    return metric if not np.isnan(metric) and metric > -1  and metric < 1 else 'undefined'

In [3]:
def compute_values_move(refexps_dir, success_dir, compound=True):
    splits = compound_splitter.load_dict('de_lower.dict')
    move_level_data = defaultdict(dict)
    move_level_align = []
    move_level_coherence = []
    success_types = set()
    playermove_level_align_speaker = defaultdict(list)
    playermove_level_coherence_speaker= defaultdict(list)

    for dialogue in listdir(refexps_dir):
        #extract dialogue type
        success_file = dialogue.replace('refexps', 'movesuccess')
        success_file = path.join(success_dir, success_file)
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue = load_data(dialogue)
        success_moves = load_data(success_file)
        #move level
        dialogue_move = move_level(dialogue)
        #iterate over moves
        for move in dialogue_move:
            refs = move[1]
            #iterate over objects
            for obj in refs:
                #iterate over objects
                refobj = refs[obj]
                #SIMILARITY OF LEXICAL CONTENT INTO
                sim_align = []
                sim_align_speaker = defaultdict(list)
                sim_coherence_speaker = defaultdict(list)
                sim_coherence = []
                for ref in refobj:
                    speaker = ref[1]
                    sim = len_div(ref, get_previous(ref, refobj), language, splits, compound) ###
                    if sim != 'undefined':
                        sim_align.append(sim)
                        sim_align_speaker[speaker].append(sim)
                sim_align = np.mean(sim_align)
                comb_ref = combinations(refobj,2)
                for comb in comb_ref:
                    if comb[0] != comb[1]:
                        sim = len_div(comb[0], comb[1], language, splits, compound) ###
                        if sim != 'undefined':
                            sim_coherence.append(sim)
                            sim_coherence_speaker[comb[0][1]].append(sim)
                            sim_coherence_speaker[comb[1][1]].append(sim)
                sim_coherence = np.mean(sim_coherence)
                #movelevel coherence and alignment (for each object)
                if not np.isnan(sim_coherence):
                    move_level_coherence.append(sim_coherence)
                if not np.isnan(sim_align):
                    move_level_align.append(sim_align)
                for s in sim_align_speaker:
                    if not np.isnan(np.mean(sim_align_speaker[s])):
                        playermove_level_align_speaker[s].append(np.mean(sim_align_speaker[s]))
                for s in sim_coherence_speaker:
                    if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                        playermove_level_coherence_speaker[s].append(np.mean(sim_coherence_speaker[s]))
            values = [sim_coherence, sim_align, sim_align_speaker['e-utts'], sim_align_speaker['p-utts'], 
                      sim_coherence_speaker['e-utts'], sim_coherence_speaker['p-utts'] ]
            move_level_data[move[0]]['values'] = values
            success = success_moves[move[0]]
            move_level_data[move[0]]['success'] = success
    move_level_align_all = []
    move_level_coherence_all = []

    playermove_level_align_speaker_all = defaultdict(list)
    playermove_level_coherence_speaker_all = defaultdict(list)

    results_move = defaultdict(list)
    
    playermove_level_coherence = []
    for s in playermove_level_coherence_speaker:
        playermove_level_coherence += playermove_level_coherence_speaker[s]
        playermove_level_coherence_speaker_all[s] += playermove_level_coherence_speaker[s]
        results_move['Move coherence '+ s] = playermove_level_coherence_speaker[s]
    results_move['Move players coherence'] = playermove_level_coherence
    results_move['Move coherence'] = move_level_coherence

    playermove_level_align = []
    for s in playermove_level_align_speaker:
        playermove_level_align += playermove_level_align_speaker[s]
        playermove_level_align_speaker_all[s] += playermove_level_align_speaker[s]
        results_move['Move alignment '+ s] = playermove_level_align_speaker[s]
    results_move['Move players alignment'] = playermove_level_align
    results_move['Move alignment'] = move_level_align

    for value in results_move:
        dataset = results_move[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return results_move

In [4]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de = compute_values_move(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_move(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_move(all_dataset_dir, all_success_dir)

/home/laura/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/laura/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


	Move coherence: 
	Mean: 0.0813921087609	Standard deviation: 0.25791568009
	Max value: 0.857142857143	Min value: -0.714285714286
	Move alignment e-utts: 
	Mean: -0.0669125391366	Standard deviation: 0.374981133702
	Max value: 0.714285714286	Min value: -0.875
	Move alignment p-utts: 
	Mean: -0.0621775446775	Standard deviation: 0.40125276648
	Max value: 0.746031746032	Min value: -0.857142857143
	Move players coherence: 
	Mean: 0.0801703598516	Standard deviation: 0.260258517663
	Max value: 0.857142857143	Min value: -0.714285714286
	Move players alignment: 
	Mean: -0.0648366901165	Standard deviation: 0.386725678246
	Max value: 0.746031746032	Min value: -0.875
	Move coherence p-utts: 
	Mean: 0.0925815308466	Standard deviation: 0.258291404237
	Max value: 0.857142857143	Min value: -0.714285714286
	Move coherence e-utts: 
	Mean: 0.0578680525813	Standard deviation: 0.262284735241
	Max value: 0.857142857143	Min value: -0.714285714286
	Move alignment: 
	Mean: -0.0878698281112	Standard deviation: 0

In [5]:
results_de_nocs = compute_values_move(de_dataset_dir, de_success_dir, compound = False)
print '\n\n'

for var in results_de:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_de_nocs[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_de_nocs[var])

	Move coherence: 
	Mean: 0.0843166845269	Standard deviation: 0.256868470532
	Max value: 0.857142857143	Min value: -0.714285714286
	Move alignment e-utts: 
	Mean: -0.0738941059569	Standard deviation: 0.366263720555
	Max value: 0.714285714286	Min value: -0.866666666667
	Move alignment p-utts: 
	Mean: -0.0572778988688	Standard deviation: 0.394321670509
	Max value: 0.746031746032	Min value: -0.857142857143
	Move players coherence: 
	Mean: 0.0825229660652	Standard deviation: 0.258318386611
	Max value: 0.857142857143	Min value: -0.714285714286
	Move players alignment: 
	Mean: -0.0666094644437	Standard deviation: 0.378910113984
	Max value: 0.746031746032	Min value: -0.866666666667
	Move coherence p-utts: 
	Mean: 0.0950247465043	Standard deviation: 0.257400530474
	Max value: 0.857142857143	Min value: -0.666666666667
	Move coherence e-utts: 
	Mean: 0.0600578377636	Standard deviation: 0.258444290843
	Max value: 0.857142857143	Min value: -0.714285714286
	Move alignment: 
	Mean: -0.0891759505602	S

In [6]:
print 'Differences between languages'
for var in results_all:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
print '\n'

print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_de['Move alignment e-utts'],results_de['Move alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'

mannwhitneyu =  stats.mannwhitneyu(results_en['Move alignment e-utts'],results_en['Move alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment en'
    print str(mannwhitneyu)+'\tSignificantly different'
                                                                        
mannwhitneyu =  stats.mannwhitneyu(results_all['Move alignment e-utts'],results_all['Move alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment all'
    print str(mannwhitneyu)+'\tSignificantly different'                                                             
                                                                        
mannwhitneyu =  stats.mannwhitneyu(results_de['Move coherence e-utts'],results_de['Move coherence p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move coherence de'
    print str(mannwhitneyu)+'\tSignificantly different'

mannwhitneyu =  stats.mannwhitneyu(results_en['Move coherence e-utts'],results_en['Move coherence p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move coherence en'
    print str(mannwhitneyu)+'\tSignificantly different'
                                                                        
mannwhitneyu =  stats.mannwhitneyu(results_all['Move coherence e-utts'],results_all['Move coherence p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move coherence all'
    print str(mannwhitneyu)+'\tSignificantly different' 

Differences between languages


Differences between players
Players move coherence de
MannwhitneyuResult(statistic=30069.5, pvalue=0.0072730725362195343)	Significantly different
Players move coherence all
MannwhitneyuResult(statistic=70754.0, pvalue=0.001350834271516433)	Significantly different


In [7]:
def compute_values_game(refexps_dir, success_dir, compound = True):
    splits = compound_splitter.load_dict('de_lower.dict')
    game_level_align = []
    game_level_coherence = []
    playergame_level_align_speaker = defaultdict(list)
    playergame_level_coherence_speaker = defaultdict(list)

    for dialogue in listdir(refexps_dir):
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = load_data(path.join(refexps_dir, dialogue))
        dialogue_game = gamerun_level(dialogue)
        for obj in dialogue_game:
            refobj = dialogue_game[obj]
            sim_align = []
            sim_align_speaker = defaultdict(list)
            sim_coherence_speaker = defaultdict(list)
            sim_coherence = []
            for ref in refobj:
                speaker = ref[1]
                sim = len_div(ref, get_previous(ref, refobj), language, splits, compound)
                if sim != 'undefined':
                    sim_align.append(sim)
                    sim_align_speaker[speaker].append(sim)
            sim_align = np.mean(sim_align)
            for s in sim_align_speaker[speaker]:
                sim_align_speaker[speaker] = np.mean(sim_align_speaker[speaker])
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = len_div(comb[0], comb[1], language, splits, compound)
                    if sim != 'undefined':
                        sim_coherence.append(sim)
                        sim_coherence_speaker[comb[0][1]].append(sim)
                        sim_coherence_speaker[comb[1][1]].append(sim)
            sim_coherence = np.mean(sim_coherence)
            #gamelevel coherence and alignment (for each object)
            if not np.isnan(sim_coherence) :
                game_level_coherence.append(sim_coherence)
            if not np.isnan(sim_align):
                game_level_align.append(sim_align)
            for s in sim_align_speaker:
                if not np.isnan(np.mean(sim_align_speaker[s])):
                    playergame_level_align_speaker[s].append(np.mean(sim_align_speaker[s]))
            for s in sim_coherence_speaker:
                if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                    playergame_level_coherence_speaker[s].append(np.mean(sim_coherence_speaker[s]))

    game_level_align_all = []
    game_level_coherence_all = []

    playergame_level_align_speaker_all = defaultdict(list)
    playergame_level_coherence_speaker_all = defaultdict(list)

    results_game = defaultdict(list)

    playergame_level_coherence = []
    for s in playergame_level_coherence_speaker:
        playergame_level_coherence += playergame_level_coherence_speaker[s]
        playergame_level_coherence_speaker_all[s] += playergame_level_coherence_speaker[s]
        results_game['game coherence '+ s] = playergame_level_coherence_speaker[s]
    results_game['game players coherence'] = playergame_level_coherence
    results_game['game coherence'] = game_level_coherence
    playergame_level_align = []
    for s in playergame_level_align_speaker:
        playergame_level_align += playergame_level_align_speaker[s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[s]
        results_game['game alignment '+ s] = playergame_level_align_speaker[s]
    results_game['game players alignment'] = playergame_level_align
    results_game['game alignment'] = game_level_align
    for value in results_game:
        dataset = results_game[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return results_game

In [8]:

results_de = compute_values_game(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_game(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_game(all_dataset_dir, all_success_dir)

	game players coherence: 
	Mean: 0.040450574686	Standard deviation: 0.212982439353
	Max value: 0.75	Min value: -0.714285714286
	game coherence e-utts: 
	Mean: 0.00825552395049	Standard deviation: 0.201387597148
	Max value: 0.714285714286	Min value: -0.714285714286
	game alignment: 
	Mean: -0.014749484656	Standard deviation: 0.204654609502
	Max value: 0.714285714286	Min value: -0.6
	game alignment p-utts: 
	Mean: 0.0129115939601	Standard deviation: 0.346958270276
	Max value: 0.714285714286	Min value: -0.857142857143
	game coherence p-utts: 
	Mean: 0.0587397819113	Standard deviation: 0.217180916909
	Max value: 0.75	Min value: -0.714285714286
	game players alignment: 
	Mean: -0.0097626877697	Standard deviation: 0.327363976699
	Max value: 0.714285714286	Min value: -0.857142857143
	game coherence: 
	Mean: 0.052970850732	Standard deviation: 0.218244834174
	Max value: 0.75	Min value: -0.714285714286
	game alignment e-utts: 
	Mean: -0.0300290811742	Standard deviation: 0.307387685154
	Max value

In [9]:
results_de_nocs = compute_values_game(de_dataset_dir, de_success_dir, compound = False)
print '\n\n'

for var in results_de:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_de_nocs[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_de_nocs[var])

	game players coherence: 
	Mean: 0.0422075139303	Standard deviation: 0.211701564152
	Max value: 0.75	Min value: -0.714285714286
	game coherence e-utts: 
	Mean: 0.00938401401862	Standard deviation: 0.19778106622
	Max value: 0.714285714286	Min value: -0.714285714286
	game alignment: 
	Mean: -0.0187871058095	Standard deviation: 0.201981353865
	Max value: 0.714285714286	Min value: -0.6
	game alignment p-utts: 
	Mean: 0.0150904853186	Standard deviation: 0.341064558487
	Max value: 0.714285714286	Min value: -0.857142857143
	game coherence p-utts: 
	Mean: 0.0608537274951	Standard deviation: 0.217016082301
	Max value: 0.75	Min value: -0.714285714286
	game players alignment: 
	Mean: -0.012672149062	Standard deviation: 0.319787027106
	Max value: 0.714285714286	Min value: -0.857142857143
	game coherence: 
	Mean: 0.0552135208572	Standard deviation: 0.218013191696
	Max value: 0.75	Min value: -0.714285714286
	game alignment e-utts: 
	Mean: -0.0374865390836	Standard deviation: 0.297306272955
	Max valu

In [10]:
print 'Differences between languages'
for var in results_all:
    ttest =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if ttest[1] < 0.05:
        print var
        print str(ttest)+'\tSignificantly different'
print '\n'

print 'Differences between players'
ttest =  stats.mannwhitneyu(results_de['game alignment e-utts'],results_de['game alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment de'
    print str(ttest)+'\tSignificantly different'

ttest =  stats.mannwhitneyu(results_en['game alignment e-utts'],results_en['game alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment en'
    print str(ttest)+'\tSignificantly different'
                                                                        
ttest =  stats.mannwhitneyu(results_all['game alignment e-utts'],results_all['game alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment all'
    print str(ttest)+'\tSignificantly different'                                                               
                                                                        
ttest =  stats.mannwhitneyu(results_de['game coherence e-utts'],results_de['game coherence p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game coherence de'
    print str(ttest)+'\tSignificantly different'

ttest =  stats.mannwhitneyu(results_en['game coherence e-utts'],results_en['game coherence p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game coherence en'
    print str(ttest)+'\tSignificantly different'
                                                                        
ttest =  stats.mannwhitneyu(results_all['game coherence e-utts'],results_all['game coherence p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game coherence all'
    print str(ttest)+'\tSignificantly different' 

Differences between languages
game players coherence
MannwhitneyuResult(statistic=42562.5, pvalue=0.045989773996959193)	Significantly different


Differences between players
Players game coherence de
MannwhitneyuResult(statistic=10234.0, pvalue=0.0017685364519441169)	Significantly different
Players game coherence en
MannwhitneyuResult(statistic=5305.5, pvalue=0.015190691199948691)	Significantly different
Players game coherence all
MannwhitneyuResult(statistic=30351.5, pvalue=7.1057132209192549e-05)	Significantly different
